In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
import data_helpers
# function: get the node name of ckpt model
# from tensorflow.python import pywrap_tensorflow
# # checkpoint_path = 'model.ckpt-xxx'
# checkpoint_path =  "../WHUCW/SDGCN/runs/Restaurants/CAtt_GCN_L2/checkpoints/model-3810"
# reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
# var_to_shape_map = reader.get_variable_to_shape_map()
# for key in var_to_shape_map:
#     print("tensor_name: ", key)

In [2]:
def preprocess(train_file, test_file):
    '''
    read from the text file.
    :return:    sen word id:[324,1413,1,41,43,0,0,0]
                sen len:[5]
                sen max len :[8]
                sen label:[0,0,1]
                target word id:[34,154,0,0]
                target len: [2]
                target max len: [4]
                targets word id :[[34,154,0,0],
                                  [34,14,12,56],
                                  [0,0,0,0]]
                targets num = 2
                targets len: [2,4,0]
                targets max num:[3]
                targets_relation_self = [[1,0,0],
                                         [0,1,0],
                                         [0.0.0]]
                targets_relation_cross = [[0,1,0],
                                          [1,0,0],
                                          [0.0.0]]
    '''
    # Data Preparation
    # ==================================================
    # Load data
    print("Loading data...")
    train_x_str, train_target_str, train_y = data_helpers.load_data_and_labels(train_file)
    dev_x_str, dev_target_str, dev_y = data_helpers.load_data_and_labels(test_file)
    test_x_str, test_target_str, test_y = data_helpers.load_data_and_labels(test_file)

    # word embedding ---> x[324,1413,1,41,43,0,0,0]  y[0,1]
    # word_id_mapping,such as  apple--->23 ,w2v  23---->[vector]
    word_id_mapping, w2v = data_helpers.load_w2v("./glove.6B.300d.txt", 300)
    max_document_length = max([len(x.split(" ")) for x in (train_x_str + dev_x_str + test_x_str)])
    max_target_length = max([len(x.split(" ")) for x in (train_target_str + dev_target_str + test_target_str)])

    # The targets  ---->[[[141,23,45],[23,45,1,2],[2]], ...]
    # The number of targets ----> [3, ...]
    train_targets_str, train_targets_num = data_helpers.load_targets(train_file)
    dev_targets_str, dev_targets_num = data_helpers.load_targets(test_file)
    test_targets_str, test_targets_num = data_helpers.load_targets(test_file)
    max_target_num = max([len(x) for x in (train_targets_str + test_targets_str)])

    # sentence ---> word_id
    train_x, train_x_len = data_helpers.word2id(train_x_str, word_id_mapping, max_document_length)
    dev_x, dev_x_len = data_helpers.word2id(dev_x_str, word_id_mapping, max_document_length)
    test_x, test_x_len = data_helpers.word2id(test_x_str, word_id_mapping, max_document_length)
    # target ---> word_id
    train_target, train_target_len = data_helpers.word2id(train_target_str, word_id_mapping, max_target_length)
    dev_target, dev_target_len = data_helpers.word2id(dev_target_str, word_id_mapping, max_target_length)
    test_target, test_target_len = data_helpers.word2id(test_target_str, word_id_mapping, max_target_length)
    # targets ---> word_id
    train_targets, train_targets_len = data_helpers.word2id_2(train_targets_str, word_id_mapping, max_target_length,
                                                              max_target_num)
    dev_targets, dev_targets_len = data_helpers.word2id_2(dev_targets_str, word_id_mapping, max_target_length,
                                                          max_target_num)
    test_targets, test_targets_len = data_helpers.word2id_2(test_targets_str, word_id_mapping, max_target_length,
                                                            max_target_num)

    # which one targets in all targets
    train_target_whichone = data_helpers.get__whichtarget(train_targets_num, max_target_num)
    test_target_whichone = data_helpers.get__whichtarget(test_targets_num, max_target_num)
    # target position
    train_target_position = data_helpers.get_position(train_file, max_document_length)
    test_target_position = data_helpers.get_position(test_file, max_document_length)

    train_targets_position = data_helpers.get_position_2(train_target_position, train_targets_num, max_target_num)
    test_targets_position = data_helpers.get_position_2(test_target_position, test_targets_num, max_target_num)

    # Relation Matrix
    # use test_target to creat the relation
    train_relation_self, train_relation_cross = data_helpers.get_relation(train_targets_num, max_target_num, 'global')
    test_relation_self, test_relation_cross = data_helpers.get_relation(test_targets_num, max_target_num, 'global')
    Train = {'x': train_x,  # int32(3608, 79)       train sentences input embeddingID
             'T': train_target,  # int32(3608, 23)       train target input embeddingID
             'Ts': train_targets,  # int32(3608, 13, 23)   train targets input embeddingID
             'x_len': train_x_len,  # int32(3608,)          train sentences input len
             'T_len': train_target_len,  # int32(3608,)          train target len
             'Ts_len': train_targets_len,  # int32(3608, 13)       train targets len
             'T_W': train_target_whichone,  # int32(3608, 13)       the ith number of all the targets
             'T_P': train_target_position,  # float32(3608, 79)
             'Ts_P': train_targets_position,  # float32(3608,13, 79)
             'R_Self': train_relation_self,  # int32(3608, 13, 13)
             'R_Cross': train_relation_cross,  # int32(3608, 13, 13)
             'y': train_y,  # int32(3608, 3)
             }
    Test = {'x': test_x,
            'T': test_target,
            'Ts': test_targets,
            'x_len': test_x_len,
            'T_len': test_target_len,
            'Ts_len': test_targets_len,
            'T_W': test_target_whichone,
            'T_P': test_target_position,
            'Ts_P': test_targets_position,
            'R_Self': test_relation_self,
            'R_Cross': test_relation_cross,
            'y': test_y,
            }
    return Train, Test
train, test = preprocess("./Restaurants_Train.txt", "./Restaurants_Test.txt")
# totext("train_", train)
# totext("test_", test)

Loading data...
(400003, 300)
400002 400003
load word-to-id done!
load word-to-id done!
load word-to-id done!
load word-to-id done!
load word-to-id done!
load word-to-id done!
load targets-to-id done!
load targets-to-id done!
load targets-to-id done!


In [3]:
def GetSDGoutput(pb_path, Train, Test):
    '''
    :param pb_path:pb?????
    :param image_path:???????
    :return:
    '''
    with tf.Graph().as_default():
        output_graph_def = tf.GraphDef()
        with open(pb_path, "rb") as f:
            output_graph_def.ParseFromString(f.read())
            tf.import_graph_def(output_graph_def, name="")
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            graph = tf.get_default_graph()
            graph_op = graph.get_operations()

            # ?????????,???????????
            # input:0??????,keep_prob:0??dropout???,?????1,is_training:0????
            # "input_x,input_x_1,input_x_2,sen_len,target_len," + \
            # "targets_all_len/Squeeze,relate_self,relate_cross," + \
            # "which_position,target_position,targets_all_position/Squeeze," + \
            # "input_y,dropout_keep_prob,output/softmax"
            input_x = sess.graph.get_tensor_by_name("input_x:0")
            input_x_1 = sess.graph.get_tensor_by_name("input_x_1:0")
            input_x_2 = sess.graph.get_tensor_by_name("input_x_2:0")
            sen_len = sess.graph.get_tensor_by_name("sen_len:0")
            target_len = sess.graph.get_tensor_by_name("target_len:0")
            relate_self = sess.graph.get_tensor_by_name("relate_self:0")
            relate_cross = sess.graph.get_tensor_by_name("relate_cross:0")
            which_position = sess.graph.get_tensor_by_name("which_position:0")
            target_position = sess.graph.get_tensor_by_name("target_position:0")
            input_y = sess.graph.get_tensor_by_name("input_y:0")
            dropout_keep_prob = sess.graph.get_tensor_by_name("dropout_keep_prob:0")
            targets_all_position = [sess.graph.get_tensor_by_name("targets_all_position/targets_all_position:0")]
            targets_all_len = [sess.graph.get_tensor_by_name("targets_all_len/targets_all_len:0")]

            # ?????????
            output = sess.graph.get_tensor_by_name("output/softmax:0")
            
            
            batches = data_helpers.batch_iter(
                list(zip(Train['x'],Train['T'],Train['Ts'],Train['x_len'], Train['T_len'], Train['Ts_len'],
                         Train['R_Self'],Train['R_Cross'],Train['T_W'],Train['T_P'],Train['Ts_P'],Train['y'])), 32, 80)
            x_batch,T_batch,Ts_batch,x_len_batch,T_len_batch,Ts_len_batch,R_Self_batch,R_Cross_batxh,T_W_batch,T_P_batch,Ts_P_batch,y_batch = zip(*batches[0])
            # feed_dict??
            feed_dict = {
                input_x: x_batch,
                input_x_1: T_batch,
                input_x_2: Ts_batch,
                sen_len: x_len_batch,
                target_len: T_len_batch,
                targets_all_len: Ts_len_batch,
                relate_self: R_Self_batch,
                relate_cross: R_Cross_batxh,
                which_position: T_W_batch,
                target_position: T_P_batch,
                targets_all_position: Ts_P_batch,
                input_y: y_batch,
                dropout_keep_prob: 0.5
            }
            return feed_dict
            # ??????
            # im = read_image(image_path, resize_height, resize_width, normalization=True)
            # im = im[np.newaxis, :]
            # ?????????????????????????????tensor?????????????
            # out=sess.run("InceptionV3/Logits/SpatialSqueeze:0", feed_dict={'input:0': im,'keep_prob:0':1.0,'is_training:0':False})

#             out = sess.run(output, feed_dict={input_text: text})
#             print("out:{}".format(out))
#             score = tf.nn.softmax(out, name='pre')
#             class_id = tf.argmax(score, 1)
#             print
#             "pre class_id:{}".format(sess.run(class_id))
GetSDGoutput("./model-8020.pb", train, test)

/root/PycharmProjects/pythonProject/data_helpers.py:82: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data = np.array(data)


TypeError: unhashable type: 'list'